# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [17]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df["Max Temp"] = city_data_df["Max Temp"] - 273.15


# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,cape town,-33.9258,18.4232,21.06,81,20,7.20,ZA,1675203842
1,1,tiksi,71.6872,128.8694,-20.38,100,100,4.88,RU,1675204207
2,2,coquimbo,-29.9533,-71.3436,19.85,77,20,5.14,CL,1675203936
3,3,yerbogachen,61.2767,108.0108,-35.57,100,76,1.26,RU,1675203697
4,4,barrow,71.2906,-156.7887,-21.99,84,0,11.83,US,1675204208


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [22]:
%%capture --no-display

# Configure the map plot
fig1=city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    size='Humidity',
    color='red',
    hover_cols=['City', 'Country', 'Humidity']
).opts(
    title='City Humidity Map',
    xlabel='Longitude',
    ylabel='Latitude',
    colorbar=True,
    height=500,
    width=800
)

# Display the map
fig1

:Scatter   [Lng]   (Lat,Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) &
(city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)].dropna()

# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna()

# Display sample data
display(ideal_city_df)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
120,120,tumut,-35.3000,148.2167,23.13,55,0,1.51,AU,1675204246
155,155,coihaique,-45.5752,-72.0662,22.97,23,0,4.12,CL,1675203616
492,492,forbes,-33.3833,148.0167,25.97,25,0,2.71,AU,1675204387


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
display(hotel_df.head())

,City,Country,Lat,Lng,Hotel Name
0,cape town,ZA,-33.9258,18.4232,
1,tiksi,RU,71.6872,128.8694,
2,coquimbo,CL,-29.9533,-71.3436,
3,yerbogachen,RU,61.2767,108.0108,
4,barrow,US,71.2906,-156.7887,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "radius": radius,
    "categories": "accommodation.hotel",
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except Exception as e:
        print(f"Error: {e}")
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cape town - nearest hotel: Townhouse Hotel
tiksi - nearest hotel: Арктика
coquimbo - nearest hotel: Hotel Iberia
Error: list index out of range
yerbogachen - nearest hotel: No hotel found
Error: 'name'
barrow - nearest hotel: No hotel found
mar del plata - nearest hotel: Nuevo Ostende
bassar - nearest hotel: Hôtel Bassar
kupang - nearest hotel: BIB & LANG GUEST HOUSE
hermanus - nearest hotel: Aloe guest house
lethem - nearest hotel: Hotel Amazonas
Error: list index out of range
lokoja - nearest hotel: No hotel found
portland - nearest hotel: the Hoxton
Error: list index out of range
leningradskiy - nearest hotel: No hotel found
mahebourg - nearest hotel: Grand Bel Air
homer - nearest hotel: Driftwood Inn & Homer Seaside Lodges
Error: list index out of range
tyup - nearest hotel: No hotel found
georgetown - nearest hotel: Page 63 hostel
narsaq - nearest hotel: Hotel Narsaq
rikitea - nearest hotel: Chez Bianca & Benoit
avarua - nearest hotel: Paradise Inn
Error: 'na

KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:

# Configure the map plot
fig = city_data_df.hvplot.scatter(
x='Lng',
y='Lat',
size='Humidity',
color='red',
hover_cols=['City', 'Country', 'Hotel Name']
)
# Display the map
fig

:Scatter   [Lng]   (Lat,Humidity,City,Country)